# Auto insurance claims

Now cars have been very popular, every car basically needs to buy insurance, so every day there are a lot of car accidents need to be assessed, and then claim.Insurance companies need to cultivate a large number of personnel to accident assessment, evaluation results hate big depends on the personal experience, the accuracy of the result of the accident assessment brings adverse impact, in order to improve the accident assessment accuracy, we need an accurate model, using the model can improve the assessment accuracy, and shorten the assessment in time, to reduce the difficulty and also reduce staff capacity requirements, and indirectly reduce the cost of insurance company personnel.

So we need to use machine learning methods based on accident history data.Simplify accident assessment through data understanding, data preprocessing and cleaning, modeling, evaluation, deployment, and other steps.